In [1]:
## import packages
from util import createGrps
from main import createGrpsV2
from groupAggMain import aggregate
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms import Predictor
from lenskit.algorithms.user_knn import UserUser
from lenskit import batch
import sys

In [2]:
## Get the training data and train the user-user collaborative filterring
# read the data (about applications)
# TODO: replace by Tine's filter  data
data = pd.read_csv('../dataset/user_with_negative_ratings_full.csv', delimiter=',')


In [3]:
## First get the groups
# parameters for group generation
size = 10
grpNum = 5
seed = 12345
pathToPickle = "../groupGenv2/user_matrix_train_pivot_01.pkl"
pathToUserMatrix = "../dataset/clean/user_ratings_neg_1000_20_20_1_train.csv"
sim_thrh = 0.8

# Choose which group generator version
version = 2

# generate synthetic groups of users and display them (Version 1)
if version == 1:
    #TODO: change '../dataset/users.tsv' to match Tine's data
    groups2 = createGrps(pd.read_csv('../dataset/users.tsv', sep='\t'), size, grpNum, seed)
else:
    groups2 = createGrpsV2(size, grpNum, pathToPickle, pathToUserMatrix, sim_thrh, seed)
display(groups2)

5


,Users
0,"[1165352, 668454, 838185, 1056891, 578645, 950..."
1,"[549231, 583914, 941975, 294558, 803135, 99662..."
2,"[410122, 277505, 692599, 783536, 1420225, 5545..."
3,"[192178, 850445, 439070, 1340921, 473967, 1327..."
4,"[988920, 485759, 481324, 1294945, 669787, 1060..."


In [4]:
# construct dataframe in format (user, item, rating) via column addition
df_ui = data.rename(columns={"UserID": "user", "JobID": "item", "Rating":"rating"})
# check data being read properly
display(df_ui.head(10)) 

# train UserUser collaborative filterring
user_user = UserUser(10, min_nbrs=3)  # Minimum (3) and maximum (10) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(df_ui)

,user,item,rating
0,47,169528,1
1,47,284009,1
2,47,2121,1
3,47,848187,1
4,47,733748,1
5,47,576958,1
6,47,262470,1
7,47,602298,1
8,47,982331,0
9,47,979937,0


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
Numba is using threading layer omp - consider TBB
found 1 potential runtime problems - see https://boi.st/lkpy-perf


In [5]:
# pd.set_option('display.max_rows', None)
all_ratings = []

## Create a User-Item matrix of scores so we can apply one of the aggregation strategies
# iterate through groups generated
for i, row in groups2.iterrows():
    # get the array of users from the row
    synGrp = row.iloc[0]

    # get recommendations for all group members
    ratings_grp = batch.recommend(recsys, synGrp, n=None,  n_jobs=1)
    
    # all_ratings contain array of users of the group and dataframe of ratings (item, score, user, rank)
    all_ratings.append((synGrp, ratings_grp))

In [6]:
num_pref = 5
limit = "sig"  # cap norm sig
strat = "misery" # add mult misery pleasure

#group_preferences_add_non = aggregate(all_ratings)
group_preferences_add_cap = aggregate(all_ratings, num_pref, limit="sig", strat='misery')
#group_preferences_mis_norm = aggregate(all_ratings, num_pref, limit="norm", strat='misery')

#display(group_preferences_add_non)
display(group_preferences_add_cap)
#display(group_preferences_mis_norm)

,Members,Recommendation
0,"[1165352, 668454, 838185, 1056891, 578645, 950...","[33567, 1030251, 1001363, 1062379, 475364]"
1,"[549231, 583914, 941975, 294558, 803135, 99662...","[467462, 758674, 764913, 175827, 805775]"
2,"[410122, 277505, 692599, 783536, 1420225, 5545...","[403015, 322071, 759309, 283718, 1815]"
3,"[192178, 850445, 439070, 1340921, 473967, 1327...","[1047062, 241110, 189081, 229937, 194305]"
4,"[988920, 485759, 481324, 1294945, 669787, 1060...","[282017, 510956, 768776, 886344, 870248]"


In [7]:
## Save the results of group aggregation to file to be later used in the evaluation
group_preferences_add_cap.to_csv("grpAggr.csv")